In [81]:
import pandas as pd 
data = pd.read_csv("/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/mergedTratado.csv", delimiter = ',',  index_col=0)
data_17_18 = pd.read_csv("/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/1718tratado.csv", delimiter = ',',  index_col=0)
data_18_19 = pd.read_csv("/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/1819tratado.csv", delimiter = ',',  index_col=0)
data_19_20 = pd.read_csv("/home/antoniolmf/Documentos/md/tp/Data-Mining-TP/1920tratado.csv", delimiter = ',',  index_col=0)

data.head()


,nationality,position,age,birth_year,value,height,position2,foot,league,games,...,Pts/G,xG,xGA,xGDiff,xGDiff/90,CL,WinCL,CLBestScorer,season_year,classificacao
player,,,,,,,,,,,,,,,,,,,,,
Burgui,ESP,"FW,MF",23.0,1993.0,1800000.0,186.0,attack - Left Winger,right,La Liga,23.0,...,1.24,39.0,53.2,-14.2,-0.37,0.0,0.0,0.0,2018,14.0
Raphaël Varane,FRA,DF,24.0,1993.0,70000000.0,191.0,Defender - Centre-Back,right,La Liga,27.0,...,2.00,83.3,45.4,37.9,1.00,1.0,1.0,0.0,2018,3.0
Rubén Duarte,ESP,DF,21.0,1995.0,2000000.0,179.0,Defender - Left-Back,left,La Liga,24.0,...,1.24,39.0,53.2,-14.2,-0.37,0.0,0.0,0.0,2018,14.0
Samuel Umtiti,FRA,DF,23.0,1993.0,60000000.0,182.0,Defender - Centre-Back,left,La Liga,25.0,...,2.45,78.3,41.1,37.2,0.98,1.0,0.0,0.0,2018,1.0
Manu García,ESP,MF,31.0,1986.0,1800000.0,183.0,midfield - Defensive Midfield,left,La Liga,30.0,...,1.24,39.0,53.2,-14.2,-0.37,0.0,0.0,0.0,2018,14.0


In [82]:
'''
colunas_numericas = data.select_dtypes(include=[float, int])

# Calcule a correlação entre as variáveis numéricas e a variável alvo
correlacao = colunas_numericas.corrwith(data['value'])

# Criar uma lista para armazenar as colunas a serem removidas
colunas_removidas = []

for coluna in colunas_numericas.columns:
    correlacao_valor = correlacao[coluna]
    #print(f"Coluna: {coluna}, Correlação: {correlacao_valor}")
    
    # Verificar se a correlação é negativa
    if correlacao_valor < 0.2:
        colunas_removidas.append(coluna)

# Remover as colunas com correlação negativa do DataFrame
data = data.drop(colunas_removidas, axis=1)
'''

'\ncolunas_numericas = data.select_dtypes(include=[float, int])\n\n# Calcule a correlação entre as variáveis numéricas e a variável alvo\ncorrelacao = colunas_numericas.corrwith(data[\'value\'])\n\n# Criar uma lista para armazenar as colunas a serem removidas\ncolunas_removidas = []\n\nfor coluna in colunas_numericas.columns:\n    correlacao_valor = correlacao[coluna]\n    #print(f"Coluna: {coluna}, Correlação: {correlacao_valor}")\n    \n    # Verificar se a correlação é negativa\n    if correlacao_valor < 0.2:\n        colunas_removidas.append(coluna)\n\n# Remover as colunas com correlação negativa do DataFrame\ndata = data.drop(colunas_removidas, axis=1)\n'

In [83]:
categorical_columns = data.select_dtypes(include=['object', 'category']).columns.tolist()

print("Categorical columns:", categorical_columns)

Categorical columns: ['nationality', 'position', 'position2', 'foot', 'league']


In [84]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

In [85]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


dropped = data.drop(['value'], axis=1)

# Select the relevant features and target variable
features_scale = dropped.drop(['nationality', 'position', 'position2', 'foot', 'league'], axis=1)
features_encode = dropped[[ 'nationality', 'position', 'position2', 'foot', 'league']]
target = data['value']

# Preprocess the scaled features
scaled_features = features_scale.apply(pd.to_numeric, errors='coerce').values
scaled_features = np.nan_to_num(scaled_features)

# One-hot encode the categorical features
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(features_encode).toarray()

# Concatenate the scaled features and encoded features
features = np.concatenate((scaled_features, encoded_features), axis=1)



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [86]:
# Build the neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

learning_rate = 0.1
optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print('Mean Squared Error:', loss)

# Make predictions
predictions = model.predict(X_test)

# Print the predicted market values
for i in range(len(predictions)):
    print('Predicted:', predictions[i][0], 'Actual:', y_test.iloc[i])

from sklearn.metrics import r2_score
# Compute R-squared score
r2_score = r2_score(y_test, predictions)
print('R-squared:', r2_score)

Epoch 1/100
45/45 [==============================] - 0s 1ms/step - loss: 253270882779136.0000
Epoch 2/100
45/45 [==============================] - 0s 1ms/step - loss: 210938594590720.0000
Epoch 3/100
45/45 [==============================] - 0s 1ms/step - loss: 186277513134080.0000
Epoch 4/100
45/45 [==============================] - 0s 1ms/step - loss: 174201172393984.0000
Epoch 5/100
45/45 [==============================] - 0s 1ms/step - loss: 150944478134272.0000
Epoch 6/100
45/45 [==============================] - 0s 1ms/step - loss: 143454793367552.0000
Epoch 7/100
45/45 [==============================] - 0s 1ms/step - loss: 132290923462656.0000
Epoch 8/100
45/45 [==============================] - 0s 1ms/step - loss: 137446729711616.0000
Epoch 9/100
45/45 [==============================] - 0s 1ms/step - loss: 132349635330048.0000
Epoch 10/100
45/45 [==============================] - 0s 1ms/step - loss: 133363792871424.0000
Epoch 11/100
45/45 [==============================] - 0s 1m

In [87]:
from sklearn.ensemble import RandomForestRegressor

# Create the RandomForestRegressor model
rf_model = RandomForestRegressor(n_estimators=200, min_samples_split= 2, min_samples_leaf=2, max_features='sqrt', max_depth=10)

rf_model.fit(X_train, y_train)

# Evaluate the models
rf_predictions = rf_model.predict(X_test)

rf_mse = mean_squared_error(y_test, rf_predictions)

print('Random Forest Mean Squared Error:', rf_mse)

# Print the predicted market values using Random Forest
for i in range(len(rf_predictions)):
    print('Random Forest - Predicted:', rf_predictions[i], 'Actual:', y_test.iloc[i])

from sklearn.metrics import r2_score
# Compute R-squared score
r2_score = r2_score(y_test, rf_predictions)
print('R-squared:', r2_score)

Random Forest Mean Squared Error: 122119685275932.72
Random Forest - Predicted: 31843792.236712337 Actual: 40000000.0
Random Forest - Predicted: 6395231.756864426 Actual: 8000000.0
Random Forest - Predicted: 16171432.127972571 Actual: 30000000.0
Random Forest - Predicted: 8229090.97031332 Actual: 8000000.0
Random Forest - Predicted: 6845443.454721976 Actual: 3000000.0
Random Forest - Predicted: 7230907.115368851 Actual: 4000000.0
Random Forest - Predicted: 16177220.862702103 Actual: 8000000.0
Random Forest - Predicted: 8786023.054355396 Actual: 9000000.0
Random Forest - Predicted: 2629583.72279554 Actual: 75000.0
Random Forest - Predicted: 16986666.189039245 Actual: 21500000.0
Random Forest - Predicted: 37694494.12518487 Actual: 65000000.0
Random Forest - Predicted: 18389379.754945215 Actual: 11000000.0
Random Forest - Predicted: 3375229.047708763 Actual: 1000000.0
Random Forest - Predicted: 10495773.960258814 Actual: 4000000.0
Random Forest - Predicted: 6948654.396411616 Actual: 50000

In [88]:
import xgboost as xgb

# Train the XGBoost model
xgb_model = xgb.XGBRegressor(subsample=1.0, n_estimators=700, max_depth=3, learning_rate=0.1, colsample_bytree=0.5, n_jobs=-1, gamma=1.0)
xgb_model.fit(X_train, y_train)

# Get feature importances
importance_scores = xgb_model.feature_importances_

# Select top-k important features
k = 40
top_k_features_indices = importance_scores.argsort()[-k:][::-1]
top_k_features = X_train[:, top_k_features_indices]

# Train and evaluate the model using the selected features
xgb_model_selected = xgb.XGBRegressor(subsample=1.0, n_estimators=700, max_depth=3, learning_rate=0.1, colsample_bytree=0.5, n_jobs=-1, gamma=1.0)
xgb_model_selected.fit(top_k_features, y_train)
xgb_predictions_selected = xgb_model_selected.predict(X_test[:, top_k_features_indices])

# Evaluate the model with selected features
xgb_mse_selected = mean_squared_error(y_test, xgb_predictions_selected)
print('XGBoost with Feature Selection Mean Squared Error:', xgb_mse_selected)

# Print the predicted market values using XGBoost with selected features
for i in range(len(xgb_predictions_selected)):
    print('XGBoost with Feature Selection - Predicted:', xgb_predictions_selected[i], 'Actual:', y_test.iloc[i])

from sklearn.metrics import r2_score
# Compute R-squared score with selected features
r2_score_selected = r2_score(y_test, xgb_predictions_selected)
print('R-squared with Feature Selection:', r2_score_selected)

XGBoost with Feature Selection Mean Squared Error: 118220311308582.75
XGBoost with Feature Selection - Predicted: 30193446.0 Actual: 40000000.0
XGBoost with Feature Selection - Predicted: 4786521.5 Actual: 8000000.0
XGBoost with Feature Selection - Predicted: 10095577.0 Actual: 30000000.0
XGBoost with Feature Selection - Predicted: 7840641.0 Actual: 8000000.0
XGBoost with Feature Selection - Predicted: 4927844.5 Actual: 3000000.0
XGBoost with Feature Selection - Predicted: 8457457.0 Actual: 4000000.0
XGBoost with Feature Selection - Predicted: 16677760.0 Actual: 8000000.0
XGBoost with Feature Selection - Predicted: 9357882.0 Actual: 9000000.0
XGBoost with Feature Selection - Predicted: 725592.7 Actual: 75000.0
XGBoost with Feature Selection - Predicted: 9482157.0 Actual: 21500000.0
XGBoost with Feature Selection - Predicted: 38525104.0 Actual: 65000000.0
XGBoost with Feature Selection - Predicted: 13239270.0 Actual: 11000000.0
XGBoost with Feature Selection - Predicted: 4028772.0 Actua

In [89]:
import lightgbm as lgb

# Train the LightGBM model
lgb_model = lgb.LGBMRegressor(n_estimators=200, learning_rate=0.1, max_depth=5)
lgb_model.fit(X_train, y_train)

# Get feature importances
importance_scores = lgb_model.feature_importances_

# Select top-k important features
k = 40
top_k_features_indices = importance_scores.argsort()[-k:][::-1]
top_k_features = X_train[:, top_k_features_indices]

# Train and evaluate the model using the selected features
lgb_model_selected = lgb.LGBMRegressor(n_estimators=200, learning_rate=0.1, max_depth=5)
lgb_model_selected.fit(top_k_features, y_train)
lgb_predictions_selected = lgb_model_selected.predict(X_test[:, top_k_features_indices])

# Evaluate the model with selected features
lgb_mse_selected = mean_squared_error(y_test, lgb_predictions_selected)
print('LightGBM with Feature Selection Mean Squared Error:', lgb_mse_selected)

# Print the predicted market values using LightGBM with selected features
for i in range(len(lgb_predictions_selected)):
    print('LightGBM with Feature Selection - Predicted:', lgb_predictions_selected[i], 'Actual:', y_test.iloc[i])

from sklearn.metrics import r2_score
# Compute R-squared score with selected features
r2_score_selected = r2_score(y_test, lgb_predictions_selected)
print('R-squared with Feature Selection:', r2_score_selected)

LightGBM with Feature Selection Mean Squared Error: 94850282440925.83
LightGBM with Feature Selection - Predicted: 34553535.789381795 Actual: 40000000.0
LightGBM with Feature Selection - Predicted: 7824881.867870362 Actual: 8000000.0
LightGBM with Feature Selection - Predicted: 18721396.75116077 Actual: 30000000.0
LightGBM with Feature Selection - Predicted: 5823517.590530969 Actual: 8000000.0
LightGBM with Feature Selection - Predicted: 4446914.465926375 Actual: 3000000.0
LightGBM with Feature Selection - Predicted: 7254750.523527697 Actual: 4000000.0
LightGBM with Feature Selection - Predicted: 10525225.233538577 Actual: 8000000.0
LightGBM with Feature Selection - Predicted: 10264099.08289246 Actual: 9000000.0
LightGBM with Feature Selection - Predicted: -214493.2491218242 Actual: 75000.0
LightGBM with Feature Selection - Predicted: 16430359.416384203 Actual: 21500000.0
LightGBM with Feature Selection - Predicted: 38124921.59871819 Actual: 65000000.0
LightGBM with Feature Selection -

In [106]:

from sklearn.model_selection import train_test_split

dropped1 = data_17_18.drop(['value'], axis=1)
dropped2 = data_18_19.drop(['value'], axis=1)
dropped3 = data_19_20.drop(['value'], axis=1)

# Select the relevant features and target variable
features_scale = dropped1.drop(['nationality', 'position', 'position2', 'foot', 'league'], axis=1)
features_encode = dropped1[[ 'nationality', 'position', 'position2', 'foot', 'league']]
target = data_17_18['value']

features_scale1 = dropped2.drop([ 'nationality', 'position', 'position2', 'foot', 'league'], axis=1)
features_encode1 = dropped2[[ 'nationality', 'position', 'position2', 'foot', 'league']]
target1 = data_18_19['value']

features_scale2 = dropped3.drop([ 'nationality', 'position', 'position2', 'foot', 'league'], axis=1)
features_encode2 = dropped3[[  'nationality', 'position',  'position2', 'foot', 'league']]
target2 = data_19_20['value']

# Preprocess the scaled features
scaled_features = features_scale.apply(pd.to_numeric, errors='coerce').values
scaled_features = np.nan_to_num(scaled_features)

scaled_features1 = features_scale1.apply(pd.to_numeric, errors='coerce').values
scaled_features1 = np.nan_to_num(scaled_features1)

scaled_features2 = features_scale2.apply(pd.to_numeric, errors='coerce').values
scaled_features2 = np.nan_to_num(scaled_features2)

# One-hot encode the categorical features
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(features_encode).toarray()

encoder1 = OneHotEncoder()
encoded_features1 = encoder1.fit_transform(features_encode1).toarray()

encoder2 = OneHotEncoder()
encoded_features2 = encoder2.fit_transform(features_encode2).toarray()

# Concatenate the scaled features and encoded features
features = np.concatenate((scaled_features, encoded_features), axis=1)

features1 = np.concatenate((scaled_features1, encoded_features1), axis=1)

features2 = np.concatenate((scaled_features2, encoded_features2), axis=1)

# Divida os dados da primeira época em conjuntos de treinamento e teste
X_train_17_18, X_test_17_18, y_train_17_18, y_test_17_18 = train_test_split(features, target, test_size= 0.6, random_state=42)

# Divida os dados da segunda época em conjuntos de treinamento e teste
X_train_18_19, X_test_18_19, y_train_18_19, y_test_18_19 = train_test_split(features1, target1, test_size=0.6, random_state=42)

# Divida os dados da terceira época em conjuntos de treinamento e teste
X_train_19_20, X_test_19_20, y_train_19_20, y_test_19_20 = train_test_split(features2, target2, test_size=0.2, random_state=42)


In [107]:
lgb_model = lgb.LGBMRegressor(n_estimators=200, learning_rate=0.1, max_depth=5)
lgb_model.fit(X_train_17_18, y_train_17_18)

importance_scores = lgb_model.feature_importances_

k = 40
top_k_features_indices = importance_scores.argsort()[-k:][::-1]
top_k_features = X_train_17_18[:, top_k_features_indices]

lgb_model_selected = lgb.LGBMRegressor(n_estimators=200, learning_rate=0.1, max_depth=5)
lgb_model_selected.fit(top_k_features, y_train_17_18)

# Ajustar as dimensões de X_test_18_19
X_test_18_19_adjusted = X_test_18_19[:, :top_k_features.shape[1]]

lgb_predictions_selected = lgb_model_selected.predict(X_test_18_19_adjusted)

lgb_mse_selected = mean_squared_error(y_test_18_19, lgb_predictions_selected)
print('LightGBM with Feature Selection Mean Squared Error:', lgb_mse_selected)

# Print the predicted market values using LightGBM with selected features
for i in range(len(lgb_predictions_selected)):
    print('LightGBM with Feature Selection - Predicted:', lgb_predictions_selected[i], 'Actual:', y_test.iloc[i])

r2_score_selected = r2_score(y_test_18_19, lgb_predictions_selected)
print('R-squared with Feature Selection:', r2_score_selected)




LightGBM with Feature Selection Mean Squared Error: 825472609811199.1
LightGBM with Feature Selection - Predicted: 56111061.62976153 Actual: 40000000.0
LightGBM with Feature Selection - Predicted: 41193952.09655349 Actual: 8000000.0
LightGBM with Feature Selection - Predicted: 26365468.109833695 Actual: 30000000.0
LightGBM with Feature Selection - Predicted: 41050363.9428344 Actual: 8000000.0
LightGBM with Feature Selection - Predicted: 30004424.991320997 Actual: 3000000.0
LightGBM with Feature Selection - Predicted: 42358000.35059779 Actual: 4000000.0
LightGBM with Feature Selection - Predicted: 26053257.392361045 Actual: 8000000.0
LightGBM with Feature Selection - Predicted: 25989106.5702414 Actual: 9000000.0
LightGBM with Feature Selection - Predicted: 43372299.524113186 Actual: 75000.0
LightGBM with Feature Selection - Predicted: 45453701.08283204 Actual: 21500000.0
LightGBM with Feature Selection - Predicted: 42808615.13154602 Actual: 65000000.0
LightGBM with Feature Selection - P